In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
dataSet = 'iono'
filePath = '/content/drive/MyDrive/Ms.C. in CSE/October 2022/CSE 6705(Meta Heuristics)/Project/iono.csv' 
data = pd.read_csv(filePath)
data = data.drop(['Unnamed: 0'], axis=1)
data 

,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8,Column_9,Column_10,...,Column_26,Column_27,Column_28,Column_29,Column_30,Column_31,Column_32,Column_33,Column_34,Column_35
0,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,0.0
1,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,1.0
2,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,0.0
3,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,1.0
4,1,0,0.02337,-0.00592,-0.09924,-0.11949,-0.00763,-0.11824,0.14706,0.06637,...,-0.01535,-0.03240,0.09223,-0.07859,0.00732,0.00000,0.00000,-0.00039,0.12011,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,1,0,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,...,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307,1.0
346,1,0,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,...,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712,1.0
347,1,0,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,...,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577,1.0
348,1,0,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,...,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243,1.0


In [4]:
import random

In [5]:
# Define the size of the population and the number of generations
POPULATION_SIZE = 10
NUM_GENERATIONS = 50


In [6]:
def get_common_value(data):
  X=data.values.tolist()
  N=len(data)
  return data,X,N

In [7]:
#determine the index of class 0 and class 1 samples
def classLabelFinding(data):
  class_0_index=[]
  class_1_index=[]
  X=data.values.tolist()
  for index in range(len(X)):
    if(X[index][34] ==1.0):
      class_1_index.append(index)
    else:
      class_0_index.append(index)
  minor_class_sample=len(class_0_index)
  major_class_sample=len(class_1_index)

  return class_0_index, class_1_index

In [8]:
import math
# Define the triplet loss function
def triplet_loss(anchor, positive, negative,data):

    # Calculate the Euclidean distance between the anchor and positive samples
    X=data.values.tolist()
    pos_dist = sum([math.dist(X[a][:34],X[p][:34]) for a, p in zip(anchor, positive)])
    # Calculate the Euclidean distance between the anchor and negative samples
    neg_dist = sum([math.dist(X[a][:34],X[n][:34]) for a, n in zip(anchor, negative)])
    # Calculate the triplet loss
    loss = max(pos_dist - neg_dist + 1, 0)
    return loss

   

In [9]:
# Define the fitness function
def fitness(individual,data):
    # Split the list into anchor, positive and negative samples
    N=len(data)
    anchor = individual[:N]
    positive = individual[N:2*N]
    negative = individual[2*N:]
    # Calculate the triplet loss for the triplet set
    loss = triplet_loss(anchor, positive, negative,data)
    return -loss  # We maximize the negative of the triplet loss

In [10]:
# Define the initialization function
def initialize_population(data):
    # Create a list of random individuals
    population = []
    N = len(data)
    X=data.values.tolist()

    class_0_index,class_1_index=classLabelFinding(data)
    
    for pop_no in range(POPULATION_SIZE):
      individual= [0 for j in range(3*N)]
      for i in range(len(X)):
        individual[i]=i
        if(X[i][34] ==1.0):
          individual[N+i]=random.choice(class_1_index)
          individual[2*N+i]=random.choice(class_0_index)
        else:
          individual[N+i]=random.choice(class_0_index)
          individual[2*N+i]=random.choice(class_1_index)
        population.append(individual)
      return population

In [11]:
# Define the selection function
def selection(population,data):
    # Select two individuals randomly from the population
    i1, i2 = random.sample(range(POPULATION_SIZE), 2)
    # Return the fitter individual
    if fitness(population[i1],data) > fitness(population[i2],data):
       
        return population[i1]
    else:
        return population[i2]

In [12]:
# Define the crossover function
def crossover(individual1, individual2):
    # Perform uniform crossover
    child = [i1 if random.random() < 0.5 else i2 for i1, i2 in zip(individual1, individual2)]
    return child

In [13]:
# Define the mutation function
def mutation(individual,data):
    # Flip a random bit with a small probability
    class_0_index,class_1_index=classLabelFinding(data)
    N=len(data)
    X=data.values.tolist()
    for i in range(len(individual)):
        if random.random() < 0.01:
         # print("1")
          if(i>=N and i<2*N):
            index=i-N
           # print("2")
            if(X[index][34] ==1.0):
              individual[N+index]=random.choice(class_1_index)
             # print("3")
            else:
              individual[N+index]=random.choice(class_0_index)
              #print("4")
          elif(i>=2*N):
            index=i-(2*N)
           # print("5")
            if(X[index][34] ==1.0):
              #print("6")
              individual[2*N+index]=random.choice(class_0_index)
            else:
              #print("7")
              individual[2*N+index]=random.choice(class_1_index)
          else:  
            pass
    return individual

In [14]:
def get_triplet_list(individual,data):
  N=len(data)
  X=data.values.tolist()
  triplet_list=[]
  for i in range(N):
    #print("i:",i)
    sample_triplet=[]
    a_index=individual[i]
    #print(a_index)
    p_index=individual[N+i]
    #print(p_index)
    n_index=individual[2*N+i]
    #print(n_index)
    anchor=X[a_index][0:34]
   # print("Anchor")
    #print(anchor)
    positive=X[p_index][0:34]
    #print("Positive")
    #print(positive)
    negative=X[n_index][0:34]
    #print("Negative")
    #print(negative)
    sample_triplet.append(anchor)
    sample_triplet.append(positive)
    sample_triplet.append(negative)
   # print(sample_triplet)
    triplet_list.append(sample_triplet)
  
  return triplet_list

In [15]:
# Define the genetic algorithm function
def genetic_algorithm(data):

    
    # Initialize the population
    population = initialize_population(data)
    fitted_population=[]
    # Iterate over the generations
    for generation in range(NUM_GENERATIONS):
        # Create a new population
        new_population = []
        # Generate offspring
        for i in range(POPULATION_SIZE):
            # Select parents
            parent1 = selection(population,data)
            parent2 = selection(population,data)
            # Crossover parents
            child = crossover(parent1, parent2)
            # Mutate child
            child = mutation(child,data)
            # Add child to new population
            new_population.append(child)
        # Replace the old population with the new population
        population = new_population
        fitness_value=[]
        for i in range(len(population)):
          fit_value=fitness(population[i],data)
          fitness_value.append(fit_value)
        max_fit=max(fitness_value)
        for i in range(len(population)):
          if(fitness_value[i]==max_fit):
            fittest_individual=population[i]
            break
            

      
        fitted_population.append(fittest_individual)
        # Find the fittest individual in the population
        #fittest_individual = max(population, key=fitness(x1,data))
        # Print the fittest individual and its fitness
        #print("Generation ", generation+1, ": ", fittest_individual, "\n Fitness:", -fitness(fittest_individual,data))
        #print("Generation ", generation+1, ": ", fittest_individual, "\n Fitness:", -fitness(fittest_individual,data),"\n Triplet_list", get_triplet_list(fittest_individual,data))
        
    return fitted_population

# **Train test spliting**

In [16]:
import numpy as np
from sklearn.model_selection import train_test_split
indices = np.arange(len(data))

y = data['Column_35']
X = data
X = X.drop(['Column_35'], axis=1)
x1, x2, y1, y2, idx1, idx2 = train_test_split(X, y, indices, test_size=0.2)

#constructing pd tables for train and test data
x1["Class"]=y1
x2["Class"]=y2
 
#making list of train and test data
# train_data=x1.values.tolist()
# test_data=x2.values.tolist()
train_data=x1
test_data=x2

In [18]:
# print(x1)

# **Final output**

In [19]:
fitted_train_triplet=genetic_algorithm(train_data)
fitted_test_triplet=genetic_algorithm(test_data)

In [20]:
from pprint import pprint
import numpy as np

In [21]:

x=get_triplet_list(fitted_train_triplet[49],train_data)

y=get_triplet_list(fitted_test_triplet[49],test_data)

In [22]:
print(len(x))
print(len(x[0]))
print(len(x[0][0]))
print(x[0])

280
3
34
[[1.0, 0.0, 1.0, 0.67784, 0.81309, 0.82021, 0.43019, 1.0, 0.20619, 0.80541, -0.43872, 1.0, -0.79135, 0.77092, -1.0, 0.40268, -0.39046, -0.58634, -0.97907, -0.42822, -0.73083, -0.76339, -0.37671, -0.97491, 0.41366, -1.0, 0.41778, -0.93296, 0.25773, -1.0, 0.9357, -0.35222, 0.98816, 0.03446], [1.0, 0.0, 0.50112, -0.03596, 0.61124, 0.01348, 0.58876, 0.01573, 0.58876, 0.02472, 0.66742, -0.00449, 0.71685, -0.04719, 0.66517, 0.00899, 0.57303, 0.02472, 0.64719, -0.07416, 0.56854, 0.14157, 0.57528, -0.03596, 0.46517, 0.04944, 0.56588, 0.00824, 0.4764, -0.03596, 0.54607, 0.10562, 0.60674, -0.0809], [1.0, 0.0, 1.0, -0.86701, 1.0, 0.2228, 0.85492, -0.39896, 1.0, -0.1209, 1.0, 0.35147, 1.0, 0.07772, 1.0, -0.14767, 1.0, -1.0, 1.0, -1.0, 0.61831, 0.15803, 1.0, 0.62349, 1.0, -0.17012, 1.0, 0.35924, 1.0, -0.66494, 1.0, 0.88428, 1.0, -0.18826]]


In [23]:
train_triplet_array=np.array(x)
test_triplet_array=np.array(y)
print(train_triplet_array)
print(test_triplet_array)
print(train_triplet_array.shape)
print(test_triplet_array.shape)

[[[ 1.       0.       1.      ... -0.35222  0.98816  0.03446]
  [ 1.       0.       0.50112 ...  0.10562  0.60674 -0.0809 ]
  [ 1.       0.       1.      ...  0.88428  1.      -0.18826]]

 [[ 0.       0.       1.      ...  1.       1.      -1.     ]
  [ 1.       0.       1.      ...  1.      -1.       1.     ]
  [ 1.       0.       0.96933 ... -0.03855  0.9842  -0.04112]]

 [[ 1.       0.       0.06404 ...  0.05419  0.07389 -0.10837]
  [ 1.       0.       0.39179 ...  0.13521  0.34411  0.12755]
  [ 1.       0.       0.65845 ...  0.76764 -0.18533  0.74356]]

 ...

 [[ 0.       0.       0.      ...  1.       0.       0.     ]
  [ 1.       0.       0.7381  ... -0.04762  0.2619   0.02381]
  [ 1.       0.       1.      ...  0.02703  1.      -0.07027]]

 [[ 1.       0.       0.90116 ... -0.05525 -0.67699 -0.19904]
  [ 1.       0.       0.66667 ...  0.2459   0.13934  0.48087]
  [ 1.       0.       0.08333 ... -1.      -1.       0.61458]]

 [[ 1.       0.       0.83367 ... -0.51206  0.64662 -0

In [25]:
np.save('icono_train.npy',train_triplet_array)
np.save('icono_test.npy',test_triplet_array)